<a href="https://colab.research.google.com/github/ananyag3110/deepfake-detection/blob/main/custom__mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import seaborn as sb
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data=pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Miniproject/mini/data.csv")
data.head()

,images_id,label
0,real_1,real
1,real_10,real
2,real_100,real
3,real_101,real
4,real_102,real


In [ ]:
height, width=128, 128
X=np.empty((data.shape[0], height, width, 3))
for i in range(data.shape[0]):
    img=load_img(r"/content/drive/MyDrive/Colab Notebooks/Miniproject/mini/{}/{}.jpg".format(data.loc[i, 'label'], \
                  data.loc[i, 'images_id']), target_size=(height, width))
    X[i]=img_to_array(img)
X.shape


In [ ]:
def changeLabels(x):
    return labels[x]

labels=data.label.unique()
labels={labels[i]:i for i in range(labels.size)}
y=data.label.apply(changeLabels)
y[:5]

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y=to_categorical(y, len(labels))
y=y.astype(int)
y[:5]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=8)
X_train.shape, y_train.shape

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=8)
X_train.shape, y_train.shape

In [ ]:
from tensorflow.keras.layers import Input, InputLayer, Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Activation, ReLU, Flatten, Dense, Add,\
    Dropout,MaxPooling2D,Concatenate,Reshape
from tensorflow.keras.models import Sequential, Model

In [ ]:
model=Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(height, width, 3)))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.45))
model.add(Dense(2, activation='softmax'))

In [ ]:
epochs=8
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
stats=model.fit(X_train, y_train, epochs=epochs, validation_split=0.2)
model.summary()

In [ ]:
stats_df=pd.DataFrame(stats.history)
stats_df['epoch']=list(range(1, epochs+1))
plot.figure(figsize=(10, 8))
sb.lineplot(y='loss', x='epoch', data=stats_df, color='deeppink', linewidth=2.5, label="Training loss")
sb.lineplot(y='val_loss', x='epoch', data=stats_df, color='darkturquoise', linewidth=2.5, label="Validation loss")
plot.grid()
plot.legend()
plot.title("Training and validation loss")

In [ ]:
stats_df=pd.DataFrame(stats.history)
stats_df['epoch']=list(range(1, epochs+1))
plot.figure(figsize=(10, 8))
sb.lineplot(y='accuracy', x='epoch', data=stats_df, color='deeppink', linewidth=2.5, label="Training accuracy")
sb.lineplot(y='val_accuracy', x='epoch', data=stats_df, color='darkturquoise', linewidth=2.5, \
            label="Validation accuracy")
plot.grid()
plot.legend()
plot.title("Training and validation accuracy")

In [ ]:
print('Accuracy:')
_, accuracy=model.evaluate(X_test, y_test)

In [ ]:
test_loss,test_acc=model.evaluate(X_test, y_test,verbose=2)
print('\n\n')
print('Test Loss',test_loss)
print('Test accuracy',test_acc)

In [ ]:
!pip install shap
import shap